In [8]:
%run '/mnt/berger2/equipes/IGE/meom/workdir/berger2/fonctions/meom_fonctions/fonctions.ipynb'

import numpy as np
import matplotlib.pyplot as plt
import scipy.io.netcdf as netcdf
import qgutils as qg
from matplotlib import ticker, cm
plt.ion()

In [9]:
#paramètres gravés dans le marbre pour le stage

Delta = 10000                               #grid spacing
f0 = 9.37456E-05                           #paramètre de Coriolis
dh = np.array([350,750,2900])              #hauteurs des couches océaniques
dh_b = np.array([(dh[0]+dh[1])/2,(dh[0]+dh[1])/2])
g_prime = np.array([2.5E-02,1.25E-02])     #gravités réduites aux interfaces
Beta = 1.75360E-11
delta_ek = 5                               #ocean bottom Ekman thickness 
nu4 = 2E09                                 #bi-harmonic viscosity
Ht = np.sum(dh)
Ht_b = np.sum(dh_b)
toc = np.array([1.31693E+01,1.81693E+01,2.41693E+01])
bf = delta_ek*f0/(2*dh[-1])

#fréquences Brunt-vaisala

N_12 = g_prime[0]/((dh[0]+dh[1])/2)
N_23 = g_prime[1]/((dh[1]+dh[2])/2)
N2 = np.array([N_12,N_23])

#densité

rho1= 1000
rho2= 1000*(1-g_prime[0]/9.81)
rho3= rho2 - 1000*g_prime[1]/9.81

rho = np.array([rho1,rho2,rho3])

In [10]:
dir0  = '/mnt/berger2/equipes/IGE/meom/workdir/berger2/qgcm-data/double_gyre_coupled/outdata_6/'
dir1 = '/mnt/berger2/equipes/IGE/meom/workdir/berger2/qgcm-data/double_gyre_coupled/outdata_9tdiff/'
file0 = 'ocpo.nc'
file1 = 'ocsst.nc'
file_at = 'atpa.nc'

In [11]:
#get fields psi
pfile_abs = dir0 + file0
pfile_rel = dir1 + file0

at_file = dir1 + file_at
#wind forcing
forcing_z = dir0 + file1

#buyoancy forcing
forcing_b_abs = dir0 + file1
forcing_b_rel = dir1 + file1

f1 = netcdf.netcdf_file(dir0 + file0,'r')
f2 = netcdf.netcdf_file(dir0 + file1,'r')


In [16]:
flot_me_et = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/tableaux_termes_abs/etapes_flot_moyen_abs.npz')
#b_me= integral_z(flot_me_et['b_me'], dh_b, Delta, Ht_b)
b_me_abs= flot_me_et['b_me']
p_me_abs = flot_me_et['p_me']

flot_me_abs = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/tableaux_termes_abs/termes_flot_moyen_abs.npz')
e_diab_abs = flot_me_abs['e_diab']


flot_me_et_rel = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/tableaux_termes_rel/etapes_flot_moyen_rel.npz')
p_me_rel = flot_me_et_rel['p_me']
b_me_rel = flot_me_et_rel['b_me']

flot_me_rel = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/tableaux_termes_rel/termes_flot_moyen_rel.npz')
u_me_rel = flot_me_rel['u_me']

diab = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/plots_exp/tableaux/termes_for_th.npz')
sst_me_abs = diab['sst_me_abs']
w_ek_me_abs = diab['w_ek_me_abs']
sst_me_rel= diab['sst_me_rel']
w_ek_me_rel= diab['w_ek_me_rel']
test_abs_sec = diab['test_abs_sec']
test_rel_sec = diab['test_rel_sec']
test_abs_prim = diab['test_abs_prim']
test_rel_prim = diab['test_rel_prim']
cov_abs_i = diab['cov_abs_i']
cov_rel_i = diab['cov_rel_i']
diab_solo_me_abs = diab['diab_solo_me_abs']
diab_solo_me_rel = diab['diab_solo_me_rel']   

In [13]:
#get xp, yp : basin size
f1 = netcdf.netcdf_file(dir0 + file0,'r')

xp = f1.variables['xp'][:-1].copy()
yp = f1.variables['xp'][:-1].copy()
zp = f1.variables['z'][:].copy()

In [14]:
N2,f0 = qg.reshape3d(dh,N2,f0)

si_t = f1.variables['time'][:].copy().size

p = qg.read_qgcm(pfile_abs, 0, var='p', rescale = 1/f0, interp = True, subtract_bc = False )
nl,N,naux = p.shape

p_at = qg.read_qgcm(at_file, 0, var='p', rescale = 1/f0, interp = True, subtract_bc = False )
nl_at,N_at,naux_at = p_at.shape

test = si_t

average = True

In [11]:
%%time

#moyenne des termes non constants

sst_me_abs = np.zeros((N,N))
w_ek_me_abs = np.zeros((N,N))
sst_me_rel = np.zeros((N,N))
w_ek_me_rel = np.zeros((N,N))

n_me = 1

for it in range(0,test):
    sst_abs = qg.read_qgcm(forcing_b_abs, it, 'sst',rescale = 1, interp=False)
    w_ek_abs = qg.read_qgcm(forcing_b_abs, it, 'wekt',rescale = 1, interp=False)
    sst_me_abs += (sst_abs-sst_me_abs)/n_me
    w_ek_me_abs += (w_ek_abs-w_ek_me_abs)/n_me
    
    sst_rel = qg.read_qgcm(forcing_b_rel, it, 'sst',rescale = 1, interp=False)
    w_ek_rel = qg.read_qgcm(forcing_b_rel, it, 'wekt',rescale = 1, interp=False)
    sst_me_rel += (sst_rel-sst_me_rel)/n_me
    w_ek_me_rel += (w_ek_rel-w_ek_me_rel)/n_me
    
    n_me +=1


CPU times: user 20.9 s, sys: 3.08 s, total: 23.9 s
Wall time: 11min 59s


In [18]:
%%time

#test importance de certains termes dans les calcul

diab_solo_me_abs = np.zeros((N,N))
diab_solo_me_rel = np.zeros((N,N))


n_me = 1

for it in range(0,test):
    
    sst_abs = qg.read_qgcm(forcing_b_abs, it, 'sst',rescale = 1, interp=False)
    w_ek_abs = qg.read_qgcm(forcing_b_abs, it, 'wekt',rescale = 1, interp=False)
    sst_rel = qg.read_qgcm(forcing_b_rel, it, 'sst',rescale = 1, interp=False)
    w_ek_rel = qg.read_qgcm(forcing_b_rel, it, 'wekt',rescale = 1, interp=False)
    
    diab_solo_me_abs += (w_ek_abs*(sst_abs-toc[0])/(toc[0]-toc[1]) - diab_solo_me_abs)/n_me
    diab_solo_me_rel += (w_ek_rel*(sst_rel-toc[0])/(toc[0]-toc[1]) - diab_solo_me_rel)/n_me
    
    
    n_me +=1


CPU times: user 15.3 s, sys: 1.48 s, total: 16.8 s
Wall time: 4min 48s


In [21]:
%%time

#test importance de certains termes dans les calcul

test_abs_sec = np.zeros((N,N))
test_rel_sec = np.zeros((N,N))
test_abs_prim = np.zeros((N,N))
test_rel_prim = np.zeros((N,N))


n_me = 1

for it in range(0,test):
    p_abs = qg.read_qgcm(pfile_abs, it, var='p', rescale = 1/f0, interp = True, subtract_bc = False)
    p_rel = qg.read_qgcm(pfile_rel, it, var='p', rescale = 1/f0, interp = True, subtract_bc = False)
    
    b_abs = qg.p2b(p_abs, dh, f0)
    b_rel = qg.p2b(p_rel, dh, f0)
    
    b_p_abs = b_abs - b_me_abs
    b_p_rel = b_rel - b_me_rel
    
    sst_abs = qg.read_qgcm(forcing_b_abs, it, 'sst',rescale = 1, interp=False)
    w_ek_abs = qg.read_qgcm(forcing_b_abs, it, 'wekt',rescale = 1, interp=False)
    sst_rel = qg.read_qgcm(forcing_b_rel, it, 'sst',rescale = 1, interp=False)
    w_ek_rel = qg.read_qgcm(forcing_b_rel, it, 'wekt',rescale = 1, interp=False)
    
    test_abs_sec += (b_p_abs[0]*w_ek_abs*(sst_me_abs-toc[0]) + b_p_abs[0]*(sst_abs - toc[0])*w_ek_me_abs
                    -test_abs_sec)/n_me
    test_rel_sec += (b_p_rel[0]*w_ek_rel*(sst_me_rel-toc[0]) + b_p_rel[0]*(sst_rel - toc[0])*w_ek_me_rel
                    -test_rel_sec)/n_me
    
    test_abs_prim += (b_p_abs[0]*w_ek_abs*(sst_abs-toc[0]) - test_abs_prim)/n_me
    test_rel_prim += (b_p_rel[0]*w_ek_rel*(sst_rel-toc[0]) - test_rel_prim)/n_me
    
    
    n_me +=1


CPU times: user 1min 5s, sys: 6.79 s, total: 1min 12s
Wall time: 20min 26s


In [35]:
np.savez('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/plots_exp/tableaux/termes_for_th.npz',
         sst_me_abs=sst_me_abs,
         w_ek_me_abs =w_ek_me_abs,
         sst_me_rel=sst_me_rel,
         w_ek_me_rel=w_ek_me_rel,
         test_abs_sec = test_abs_sec,
         test_rel_sec = test_rel_sec,
         test_abs_prim = test_abs_prim,
         test_rel_prim = test_rel_prim,
         cov_abs_i = cov_abs_i,
         cov_rel_i = cov_rel_i)


In [19]:
np.savez('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/Lorenz_cycle/plots_exp/tableaux/termes_for_th2.npz',
         diab_solo_me_abs = diab_solo_me_abs,
         diab_solo_me_rel = diab_solo_me_rel)